保险场景 VLM使用

In [4]:
import os
from openai import OpenAI
import pandas as pd

client = OpenAI(
    api_key="sk-882e296067b744289acf27e6e20f3ec0", 
    base_url="https://dashscope.aliyuncs.com/compatible-mode/v1",
)

# 调用VLM，得到推理结果
# user_prompt：用户想要分析的内容
# image_url：想要分析的图片
def get_response(user_prompt, image_url):
    # 得到image_url_list，一张图片也放到[]中
    if image_url.startswith('[') and ',' in image_url:
        # 属于image_url list
        image_url = image_url.strip()
        image_url = image_url[1:-1]
        image_url_list = image_url.split(',')
        image_url_list = [temp_url.strip() for temp_url in image_url_list]
    else:
        image_url_list = [image_url]
    # 得到messages
    content = [{"type": "text", "text": f"{user_prompt}"}]
    for temp_url in image_url_list: 
        image_url = f"https://vl-image.oss-cn-shanghai.aliyuncs.com/{temp_url}.jpg"
        content.append({"type": "image_url","image_url": {"url": f"{image_url}"}})
    messages=[{
                "role": "user",
                "content": content
            }
        ]

    print(f'messages={messages}')
    completion = client.chat.completions.create(
        model="qwen-vl-max-2025-04-08", #qwen-vl-plus
        messages=messages    
        )
    #print(completion.model_dump_json())
    return completion

In [5]:
df = pd.read_excel('./prompt_template_cn.xlsx')
df['response'] = ''
for index, row in df.iterrows():
    user_prompt = row['prompt']
    image_url = row['image']
    # 得到VLM推理结果
    completion = get_response(user_prompt, image_url)
    response = completion.choices[0].message.content
    df.loc[index, 'response'] = response
    print(f"{index+1} {user_prompt} {image_url}")
df.to_excel('./prompt_template_cn_result-20250430.xlsx', index=False)

messages=[{'role': 'user', 'content': [{'type': 'text', 'text': '你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。'}, {'type': 'image_url', 'image_url': {'url': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/1-vehicle-odometer-reading.jpg'}}]}]
1 你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。 1-vehicle-odometer-reading
messages=[{'role': 'user', 'content': [{'type': 'text', 'text': '你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。'}, {'type': 'image_url', 'image_url': {'url': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/2-vehicle-odometer-reading.jpg'}}]}]
2 你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。 2-vehicle-odometer-reading
messages=[{'role': 'user', 'content': [{'type': 'text', 'text': '你是一名汽车保险承保专家。这里有一组多角度拍摄的车辆照片，请根据这些照片分析关键的承保信息。'}, {'type': 'image_url', 'image_url': {'url': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/3-vehicle-underwriting-1.jpg'}}, {'type': 'image_url', 'image_url': {'url': 'https://vl-image.oss-cn-shanghai.aliyuncs.com/3-vehicle-underwriting-2.jpg'}}, {'type': 'image_url', 'image_url': {'ur

In [6]:
df

,id,prompt,image,response
0,1,你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。,1-vehicle-odometer-reading,从这张车辆里程表的图片中，我们可以提取以下关键信息：\n\n1. **当前速度**：\n ...
1,2,你是一名汽车保险承保专家。这里有一张车辆里程表的图片。请从中提取关键信息。,2-vehicle-odometer-reading,从这张车辆里程表的图片中，我们可以提取以下关键信息：\n\n1. **当前速度**：\n ...
2,3,你是一名汽车保险承保专家。这里有一组多角度拍摄的车辆照片，请根据这些照片分析关键的承保信息。,"[3-vehicle-underwriting-1, 3-vehicle-underwrit...",根据提供的多角度拍摄的车辆照片，我们可以分析出以下关键的承保信息：\n\n### 1. **...
3,4,你是一名汽车保险的风险监控专家。这里有一张车内摄像头的照片，请判断是否有危险行为。,4-Dangerous-driving-behavior-detection,根据提供的车内摄像头照片，可以观察到以下几点来判断是否有危险行为：\n\n1. **驾驶员的...
4,5,你是一名汽车保险的风险监控专家。这里有一段来自行车记录仪的视频片段，请简要分析配备此行车记录...,5-Dangerous-driving-behavior-detection,要分析配备此行车记录仪的车辆是否存在任何危险或鲁莽驾驶行为，我们首先需要仔细观察视频片段中的...
5,6,你是一名汽车保险的风险监控专家。这里有一段来自行车记录仪的视频片段，请简要分析配备此行车记录...,"[6-Dangerous-driving-behavior-detection-1, 6-D...",为了分析配备此行车记录仪的车辆是否存在任何危险或鲁莽驾驶行为，我们可以通过以下步骤进行评估：...
6,7,您是一名汽车保险理赔处理专家。这里有一张车祸现场的照片，请简要评估损坏情况。,7-vehicle-damage-evaluation,根据提供的车祸现场照片，以下是损坏情况的简要评估：\n\n### 1. **黑色轿车的损坏情...
7,8,您是一名汽车保险理赔处理专家。这里有一张车祸现场的照片，请简要评估损坏情况。,8-vehicle-damage-evaluation,根据提供的照片，以下是对该车祸现场车辆损坏情况的简要评估：\n\n### 1. **车身划痕...
8,9,你是一名汽车保险理赔处理专家。这里有一张车祸现场的照片。请以JSON格式提取以下信息：道路状...,9-extraction-of-auto-accident-elements,"```json\n{\n ""道路状况"": ""湿滑"",\n ""天气状况"": ""阴天"",\n..."
9,10,你是一名汽车保险理赔处理专家。这里有一张车祸现场的照片。请以JSON格式提取以下信息：道路状...,10-extraction-of-auto-accident-elements,"```json\n{\n ""道路状况"": ""干燥"",\n ""天气状况"": ""晴朗"",\n..."
